In [2]:
# Dependencies
from splinter import Browser
from requests import get
import re
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import select
import urllib.request
from bs4 import BeautifulSoup
import requests
import pymongo

In [3]:
# r = requests.get(url)
# soup = BeautifulSoup(r.text, 'html.parser')
# print(soup)
executable_path = {"executable_path": "chromedriver.exe"}
url = 'https://twitchtracker.com/channels/viewership'

In [4]:
driver = webdriver.Chrome()
driver.get(url)

In [5]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
# print(soup)

In [6]:
pd.read_html(html)[0]
df_tracker_table = pd.read_html(html)[0]
df_tracker_table.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,AVG Viewers,Time Streamed,All Time Peak Viewers,Hours Watched,Rank,Followers Gained,Total Followers,Total Views
0,#1,NaN,tommyinnit,256800,23.5hours,650237,6026250,1,+796301,3420936,31248801
1,#2,NaN,auronplay,168547,83.4hours,313347,14056783,2,+732256,7177886,100296217
2,#3,NaN,QuackityHQ,122926,30hours,232299,3687788,5,+531103,2258810,13873969
3,#4,NaN,karljacobs,110698,21.7hours,206847,2400306,9,+306022,1451026,9249417
4,#5,NaN,GeorgeNotFound,93375,13.1hours,140629,1218549,13,+329014,2249633,8452647


In [7]:
df_tracker_table = df_tracker_table[['Unnamed: 2', 
                                     'AVG Viewers', 
                                     'Time Streamed', 
                                     'All Time Peak Viewers',
                                     'Hours Watched', 
                                     'Rank', 
                                     'Followers Gained', 
                                     'Total Followers', 
                                     'Total Views']]
# df_tracker_table

In [8]:
df_tracker_table_cleaner = df_tracker_table.drop([10], axis='index')
df_tracker_table_cleaner

,Unnamed: 2,AVG Viewers,Time Streamed,All Time Peak Viewers,Hours Watched,Rank,Followers Gained,Total Followers,Total Views
0,tommyinnit,256800,23.5hours,650237,6026250,1,+796301,3420936,31248801
1,auronplay,168547,83.4hours,313347,14056783,2,+732256,7177886,100296217
2,QuackityHQ,122926,30hours,232299,3687788,5,+531103,2258810,13873969
3,karljacobs,110698,21.7hours,206847,2400306,9,+306022,1451026,9249417
4,GeorgeNotFound,93375,13.1hours,140629,1218549,13,+329014,2249633,8452647
5,Rubius,89529,84.7hours,344676,7580098,8,+484614,8062966,136116803
6,BLASTPremier,88628,55.3hours,208744,4896707,10,+114755,788302,61795246
7,ibai,84508,114.9hours,533239,9707099,6,+590122,4955236,140846760
8,TheGrefg,78109,169.7hours,2470347,13256476,4,+1095118,6569930,107981887
9,xQcOW,76890,326.8hours,222720,25128843,3,+1773138,7519357,266566406


In [9]:
df_tracker_table_cleaner.rename(columns= {
                            'Unnamed: 2' : 'streamer_handle',
                            'AVG Viewers' : 'avg_viewers',
                            'Time Streamed' : 'time_streamed(hr)',
                            'All Time Peak Viewers' : 'peak_viewers',
                            'Hours Watched' : 'hours_watched',
                            'Rank' : "rank",
                            'Followers Gained' : 'followers_gained(thou)', 
                            'Total Followers' : 'total_followers',
                            'Total Views' : 'total_views(mil)'
                            
}, inplace=True)
df_tracker_table_cleaner.head()

,streamer_handle,avg_viewers,time_streamed(hr),peak_viewers,hours_watched,rank,followers_gained(thou),total_followers,total_views(mil)
0,tommyinnit,256800,23.5hours,650237,6026250,1,+796301,3420936,31248801
1,auronplay,168547,83.4hours,313347,14056783,2,+732256,7177886,100296217
2,QuackityHQ,122926,30hours,232299,3687788,5,+531103,2258810,13873969
3,karljacobs,110698,21.7hours,206847,2400306,9,+306022,1451026,9249417
4,GeorgeNotFound,93375,13.1hours,140629,1218549,13,+329014,2249633,8452647


In [11]:
# Clean up columns
df_tracker_table_cleaner['time_streamed(hr)'] = df_tracker_table_cleaner['time_streamed(hr)'].str.replace(r'hours', '')

df_tracker_table_cleaner['followers_gained(thou)'] = df_tracker_table_cleaner['followers_gained(thou)'].str.replace(r'+', '')

df_tracker_table_cleaner

,streamer_handle,avg_viewers,time_streamed(hr),peak_viewers,hours_watched,rank,followers_gained(thou),total_followers,total_views(mil)
0,tommyinnit,256800,23.5,650237,6026250,1,796301,3420936,31248801
1,auronplay,168547,83.4,313347,14056783,2,732256,7177886,100296217
2,QuackityHQ,122926,30,232299,3687788,5,531103,2258810,13873969
3,karljacobs,110698,21.7,206847,2400306,9,306022,1451026,9249417
4,GeorgeNotFound,93375,13.1,140629,1218549,13,329014,2249633,8452647
5,Rubius,89529,84.7,344676,7580098,8,484614,8062966,136116803
6,BLASTPremier,88628,55.3,208744,4896707,10,114755,788302,61795246
7,ibai,84508,114.9,533239,9707099,6,590122,4955236,140846760
8,TheGrefg,78109,169.7,2470347,13256476,4,1095118,6569930,107981887
9,xQcOW,76890,326.8,222720,25128843,3,1773138,7519357,266566406


In [16]:
df_tracker_table_cleaner.to_csv('top_twenty_streamers_in_last_thirty_days.csv')

In [ ]:
# Below is disregarded code....oh well

In [ ]:
# Empty lists to store data
streamer_handle = []
avg_views = []
time_streamed = []
peak_viewers= []
hours_watched = []
rank_value = []
total_views = []

In [ ]:
# Parse through each tag and pull out text info
channel = soup.find_all('a', {'style' : 'color:inherit'})
views = soup.find_all('td', {'class' : 'color-viewers'})
time = soup.find_all('td', {'class' : 'color-streamed'})
peak_views = soup.find_all('td', {'class' : 'color-viewersMax'})
hours = soup.find_all('td', {'class' : 'color-watched'})
rank = soup.find_all('span', {'class' : 'rank-value'})
all_views = soup.find_all('td', {'class' : 'color-views'})

In [ ]:
for streamer in rank:
    rank_value.append(streamer.get_text())
print(rank_value)

In [ ]:
for streamer in channel:
    streamer_handle.append(streamer.get_text())
print(streamer_handle)

In [ ]:
for streamer in all_views:
    total_views.append(streamer.get_text())
print(total_views)

In [ ]:
for streamer in views:
    avg_views.append(streamer.get_text())
print(avg_views)


In [ ]:
for streamer in time:
    time_streamed.append(streamer.get_text())

print(time_streamed)

In [ ]:
for streamer in peak_views:
    peak_viewers.append(streamer.get_text())
print(peak_viewers)

In [ ]:
for streamer in hours: 
    hours_watched.append(streamer.get_text())
print(hours_watched)

In [ ]:
tracker_df = {'streamer_handle' : streamer_handle,
              'average_viewers':avg_views,
              'hrs_streamed': time_streamed,
              'all_time_peak': peak_viewers, 
              'hrs_watched': hours_watched,
              'rank' : rank_value, 
              'total_views_mil' : total_views}
# tracker_df


In [ ]:
# Put the combined lists into a dataframe
complete_tracker_df = pd.DataFrame.from_dict(tracker_df, orient='index')
complete_tracker_df = complete_tracker_df.transpose()
# complete_tracker_df

In [ ]:
# Clean up columns
complete_tracker_df['hrs_streamed'] = complete_tracker_df['hrs_streamed'].str.replace(r'hours', '')
complete_tracker_df['hrs_watched'] = complete_tracker_df['hrs_watched'].str.replace(r'M', '')
complete_tracker_df['total_views_mil'] = complete_tracker_df['total_views_mil'].str.replace(r'M', '')
complete_tracker_df